In [1]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage
! pip install xlrd
! pip install openpyxl
 

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
## import os
## os.getcwd()
## os.chdir('/Users/joshuacalianos/Documents/qss20_finalproject')

'/Users/joshuacalianos/Documents/qss20_finalproject/QSS20_TEXTASDATA_PROJ'

In [8]:
# load data
addendum = pd.read_csv("FOIA_2021-F-05932_raw_data_combined.csv")
disclosure = pd.read_csv ("H-2A_Disclosure_Data_FY_combined.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17,18,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
addendum.head()
disclosure.head()

16783

,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS
0,H-300-19274-066174,Meal Provision,E.1,Employer will furnish free and convenient cook...
1,H-300-19274-066174,Job Requirements,B.6,Three (3) months experience with references re...
2,H-300-19274-066174,Daily Transportation,F.1,Living & laundry facilities available. Housing...
3,H-300-19274-066174,Job Requirements,B.6,Should the Employers worker’s compensation ins...
4,H-300-19274-066174,Daily Transportation,F.1,The employer shall provide transportation in t...


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20265-835437,Determination Issued - Certification,2020/9/24,2020/9/30,Individual Employer,Y,Seasonal,N,Benito Martinez Jr.,NaN,...,N,1.0,N,$12.68,$12.68,$55.00,14073199505,benitomartinez863@gmail.com,NaN,0.0
1,H-300-20260-827678,Determination Issued - Certification,2020/9/16,2020/9/24,Association - Joint Employer,N,Seasonal,N,WESTERN RANGE ASSOCIATION,NaN,...,N,1.0,N,NaN,$12.68,$55.00,12085952226,applicants@westernrange.net,NaN,0.0
2,H-300-20260-827308,Determination Issued - Certification,2020/9/16,2020/9/24,Association - Agent,N,Seasonal,N,Allen Ranches LLC,NaN,...,N,1.0,N,NaN,$12.68,$55.00,13074722105,info@mpaswy.com,NaN,0.0
3,H-300-20260-826330,Determination Issued - Certification,2020/9/18,2020/9/30,Individual Employer,N,Seasonal,N,Steven Edge,NaN,...,N,1.0,N,NaN,$12.68,$55.00,18433407334,sdedge@sccoast.net,NaN,0.0
4,H-300-20259-823987,Determination Issued - Withdrawn,2020/9/18,2020/9/29,Individual Employer,N,Seasonal,N,Floyd Frey,NaN,...,N,1.0,N,NaN,$12.68,$55.00,14062650953,floydfrey@yahoo.com,NaN,0.0


In [19]:
#Data Cleaning
print(len(addendum.index))

spanish_words = ['COMPENSACIÓN', 'OCUPACIÓN', 'PAGO', 'TRANSPORTE', 'REGLAS', 'CONTRATO', 'HORAS'
                'TRABAJADORES', 'PARTE', 'TARIFAS', 'CUARTOS', 'TRABAJO', 'REGISTROS', 'ADICIONALES',
                'COMIDAS', 'EMPLEO', 'TERMINACIÓN', 'VIVIENDA', 'EMPLEADOR', 'ADICIONALES', 'FRECUENCIA',
                'PRODUCCIÓN', 'LIQUIDACIÓN', 'Comida', 'HERRAMIENTAS', 'PRUEBA', 'DIARIA', 'TARIFAS',
                'PARADO', 'ADICIONALES', 'COBERTURA', 'PROPORCIONADOS', 'PARTE', 'ABANDONO', 'DEDUCCIONES',
                'CAUSA', 'GARANTÍAS', 'DIVULGACIÓN', 'COMIDAS', 'POLÍTICA', 'DISCIPLINA', 'DESPIDOS', 'Derechos',
                'IMPOSIBILIDAD', 'DETALLES', 'DIVULGACIÓN', 'BLOQUEO', 'ABANDONO', 'GANANCIAS', 'COBERTURA', 'PEDIDOS',
                'PAGO', 'Oportunidad', 'Trabajo']

spanish_to_remove = "|".join(spanish_words)
print(spanish_to_remove)
temp = addendum[addendum.SECTION_NAME.str.contains(fr'{spanish_to_remove}', na = False, regex=True, case=False)]
print(temp.SECTION_NAME.sample(10))

print(len(temp.index))

102317
COMPENSACIÓN|OCUPACIÓN|PAGO|TRANSPORTE|REGLAS|CONTRATO|HORASTRABAJADORES|PARTE|TARIFAS|CUARTOS|TRABAJO|REGISTROS|ADICIONALES|COMIDAS|EMPLEO|TERMINACIÓN|VIVIENDA|EMPLEADOR|ADICIONALES|FRECUENCIA|PRODUCCIÓN|LIQUIDACIÓN|Comida|HERRAMIENTAS|PRUEBA|DIARIA|TARIFAS|PARADO|ADICIONALES|COBERTURA|PROPORCIONADOS|PARTE|ABANDONO|DEDUCCIONES|CAUSA|GARANTÍAS|DIVULGACIÓN|COMIDAS|POLÍTICA|DISCIPLINA|DESPIDOS|Derechos|IMPOSIBILIDAD|DETALLES|DIVULGACIÓN|BLOQUEO|ABANDONO|GANANCIAS|COBERTURA|PEDIDOS|PAGO|Oportunidad|Trabajo
71787                      Job Duties - FRECUENCIA DE PAGO
51771              Job Duties - IMPOSIBILIDAD DEL CONTRATO
48141                   Job Duties - GARANTÍA TRES CUARTOS
20115               Job Duties - La Oportunidad de Trabajo
60082    Job Duties - REGLAS DE LA OCUPACIÓN DE LA VIVI...
17284    Job Duties - Programa por Comida Justa y Derec...
88133                   Job Duties - GARANTÍA TRES CUARTOS
79286    Job Duties - ABANDONO DE EMPLEO O TERMINACIÓN ...
45126    Job

In [22]:
# Dropping duplicates in addenda

## perhaps we want to create a priority order for which section we should keep if the
## same data is inputted in multiple sections?

len(addendum)
addendum = addendum.drop_duplicates(["CASE_NUMBER", "SECTION_DETAILS"])
len(addendum)


102317

99620

In [45]:
## collapsing identical case/sections into one listing each

## code modified from fifth answer at https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
## because original groupby was including a bunch of linebreaks


addendum.sort_values(["CASE_NUMBER", "SECTION_NUMBER"])

addendum["SECTION_DETAILS"].to_string()

addendum = addendum.groupby(['CASE_NUMBER','SECTION_NAME', "SECTION_NUMBER"])['SECTION_DETAILS'].apply(lambda text: ''.join(text.to_string(index=False))).str.replace('(\\n)', '').reset_index()



,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS
23,H-300-19274-066154,Pay Deductions,A.11,Reasonable repair cost of damage from delibera...
16,H-300-19274-066154,Job Requirements,B.6,Operating large farm tractors and equipment as...
18,H-300-19274-066154,Job Requirements,B.6,Should the Employers worker’s compensation ins...
19,H-300-19274-066154,Job Requirements,B.6,Three (3) months experience required for the j...
22,H-300-19274-066154,Meal Provision,E.1,Employer will furnish free and convenient cook...
...,...,...,...,...
101081,H-300-20356-972294,Job Requirements - Housing Rules 10-25,B.6,10.\tWorkers living in employer's housing may ...
101147,H-300-20356-972294,Job Requirements - A6/ Anticipated days and ho...,B.6,Anticipated Hours of Work: Worker will report ...
101274,H-300-20366-985558,Job Duties - OTHER CLARIFICATIONS AND ASSURANC...,A.8a,OTHER CLARIFICATIONS AND ASSURANCES: \n\nWORKE...
101421,H-300-20366-985558,Job Duties - OTHER CLARIFICATIONS AND ASSURANC...,A.8a,TERMS AND CONDITION CHANGES: The employer wil...


<ipython-input-45-10398dd289e4>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  addendum = addendum.groupby(['CASE_NUMBER','SECTION_NAME', "SECTION_NUMBER"])['SECTION_DETAILS'].apply(lambda text: ''.join(text.to_string(index=False))).str.replace('(\\n)', '').reset_index()


In [52]:
print(addendum["SECTION_DETAILS"][2])
len(addendum)

addendum = addendum.rename(columns = {"SECTION_DETAILS":"JOB_DESCRIPTION"})



Operating large farm tractors and equipment as ...Should the Employers worker’s compensation insu...Three (3) months experience required for the jo...


82867

In [ ]:
#Data Aggregation

In [11]:
# merging testing code (fully applicable after data cleaning and the necessary steps above)
## left join  
merged_data = pd.merge(disclosure,
                             addendum,
                             left_on = 'CASE_NUMBER',
                             right_on = 'CASE_NUMBER',
                             how = "left",
                             indicator = "case_merge_status")
merged_data.case_merge_status.value_counts()
## merged_data.shape()


both          102204
left_only       3255
right_only         0
Name: case_merge_status, dtype: int64

TypeError: 'tuple' object is not callable

In [12]:
# ## look at one that doesnt have job 
merged_data.loc[merged_data.case_merge_status == "left_only",
                  ['CASE_NUMBER','SECTION_DETAILS',"EMPLOYER_STATE"]].sample(50)

,CASE_NUMBER,SECTION_DETAILS,EMPLOYER_STATE
26248,H-300-20097-464281,NaN,CO
87902,H-300-19262-498320,NaN,CO
79365,H-300-19330-167014,NaN,MT
60625,H-300-20010-245780,NaN,CA
87909,H-300-19262-322660,NaN,CO
55501,H-300-20018-262790,NaN,WI
40553,H-300-20052-339997,NaN,TN
92696,H-300-20261-830665,NaN,TN
87937,H-300-19261-016855,NaN,NE
92085,H-300-20265-836315,NaN,SC


In [13]:
# merging testing code (fully applicable after data cleaning and the necessary steps above)
## inner join  
merged_data_inner = pd.merge(addendum,
                             disclosure,
                             left_on = 'CASE_NUMBER',
                             right_on = 'CASE_NUMBER',
                             how = "inner",
                             indicator = "case_merge_status")
merged_data_inner.case_merge_status.value_counts()


both          102204
left_only          0
right_only         0
Name: case_merge_status, dtype: int64

In [14]:
merged_data.loc[merged_data.case_merge_status == "both",
                  ['CASE_NUMBER','SECTION_DETAILS',"EMPLOYER_STATE"]].sample(50)

,CASE_NUMBER,SECTION_DETAILS,EMPLOYER_STATE
24068,H-300-20108-494746,REQUIRED DEPARTURE: H-2A workers must depart t...,CA
76068,H-300-19340-184799,The Worker may never ride on agricultural equi...,WA
97161,H-300-20314-904616,Due to the need to renew Worker?s Compensation...,ID
6464,H-300-20192-707173,Workers will be required to prepare farm land ...,MI
70204,H-300-19353-208905,Please contact the employer point of contact ...,AR
42257,H-300-20049-329494,EQUIPMENT OPERATOR: General farm equipment ope...,MI
64085,H-300-20003-230973,Due to the need to renew Worker’s Compensation...,MT
32758,H-300-20078-415172,Employer guarantees to provide workers with ho...,CO
35979,H-300-20065-382752,The Workers who do not complete 50% of the con...,OR
69366,H-300-19354-210176,. In this case the employer will only pay for...,FL


In [ ]:
# topic modeling code (fully applicable after data cleaning and the necessary steps above)
